The following cell will install kohya_ss on your machine. 

Choose the following answers in the console window:
    
* NO
* Torch v. 1.

(long pause)

* This machine
* No distributed training
* No
* No
* No
* all
* No

In [1]:
from kaia.infra import ConsoleExecutor, Loc

ENV_NAME = 'kaia-kohya-ss'
FOLDER_NAME = 'kohya-ss'

ConsoleExecutor.wait = True

if not Loc.get_python_by_env(ENV_NAME).is_file():
    ConsoleExecutor.execute(f'conda remove --name {ENV_NAME} --all -y')
    ConsoleExecutor.execute(f'conda create --name {ENV_NAME} python=3.10 -y')
    
kohya_folder = Loc.root_folder.parent/FOLDER_NAME

if not kohya_folder.is_dir():
    ConsoleExecutor.execute(f'git clone https://github.com/bmaltais/kohya_ss.git {kohya_folder}')
    ConsoleExecutor.execute(
        f'call {Loc.conda_path} activate {ENV_NAME}\n{kohya_folder/"setup.bat"}',
        kohya_folder
    )

Let's prepare the folders:

In [2]:
from kaia.infra import Loc
import os
import shutil

model_name = 'dv'

lora_folder = Loc.data_folder/'lora'/model_name
logs_folder = lora_folder/'logs'
os.makedirs(logs_folder, exist_ok=True)
output_folder = lora_folder/'output'
os.makedirs(output_folder, exist_ok=True)
images_folder =  lora_folder/'images' 

And now the data:

In [3]:
from pathlib import Path
from zoo.images.tools.annotator import ImageTagAnnotator


basepath = Path('samples')
annotation_folder = basepath/'annotations'
extraction_folder = basepath/'extraction'
ImageTagAnnotator.export('annotation.parquet', extraction_folder, annotation_folder,images_folder)

`kohya_ss` actually allows training from the command line, so we don't need to run it as a server, we only need to prepare command line arguments. First, define the constans:

In [4]:
from zoo.images.tasks import Automatic1111

#Model from Automatic1111.location/'models/Stable-diffusion' folder
BASE_MODEL = 'AOM3_orangemixs.safetensors'

#Pick the maximal batch size that fits to your VRAM
BATCH_SIZE = 3

args=f'''
{kohya_folder/'venv/scripts/python'}
"{kohya_folder/'train_network.py'}"
--enable_bucket 
--pretrained_model_name_or_path="{Automatic1111.location/'models/Stable-diffusion'/BASE_MODEL}" 
--train_data_dir="{images_folder}" 
--resolution=512,512
--output_dir="{output_folder}" 
--logging_dir="{logs_folder}"
--network_alpha="1" 
--save_model_as=safetensors 
--network_module=networks.lora 
--text_encoder_lr=5e-05 
--unet_lr=0.0001 
--network_dim=8 
--output_name="{model_name}"
--lr_scheduler_num_cycles="1"
--learning_rate="0.0001" 
--lr_scheduler="cosine" 
--train_batch_size="{BATCH_SIZE}"
--save_every_n_epochs="1" 
--mixed_precision="fp16" 
--save_precision="fp16" 
--cache_latents 
--optimizer_type="AdamW8bit" 
--max_data_loader_n_workers="0" 
--bucket_reso_steps=64 
--xformers 
--bucket_no_upscale
'''

args = args.replace('\n', ' ')

Now we can run LORA training. This will take awhile, depending on the hardware you have:

In [5]:
#ConsoleExecutor.execute(args, kohya_folder)

Now we copy the output of the training in Automatic1111 folder:

In [6]:
import shutil

shutil.copy(output_folder/f'{model_name}.safetensors', Automatic1111.location/'models/Lora')
pass

And now we can generate the photos in Stable Diffusion:

In [7]:
Automatic1111.run_if_absent()

In [8]:
from kaia.infra.tasks import TqdmProgressReporter
from zoo.images.tools import ConvertImage
from zoo.images.tasks import Auto1TextToImage

prompt = ' <lora:dv:0.9> dv playing guitar on the rock concert'
negative = ''

task = Auto1TextToImage()
reporter = TqdmProgressReporter()
image = task.run(
    reporter, 
    {
        'prompt': prompt, 
        'negative_prompt': negative,
        'enable_hr': False,
        'batch_size': 4,
        'enable_hr': False
    }
)

ConvertImage(image).resize(200,200).to_pywidget()

  0%|          | 0/100 [00:00<?, ? %/s]